<a href="https://colab.research.google.com/github/Maxxx-VS/The-Founder/blob/master/27_4_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Определитесь, какую задачу будет решать ваша нейронная сеть.
2. Продумайте интерфейс взаимодействия с пользователем, какими параметрами модели пользователь будет управлять.
3. Обучите модель на любом публичном датасете или возьмите из любого предыдущего урока. Вспомните как происходит загрузка и выгрузка моделей в Keras.
4. Загрузите обученную модель в Colab с интерфейсом (деплой модели).
5. Создайте интерфейс для инференса вашей модели (для запросов к модели).
6. Изучите как происходит загрузка файлов для моделей с помощью Streamlit по [ссылке](https://docs.streamlit.io/develop/api-reference/widgets/st.file_uploader).
7. Добавьте в интерфейс возможность загрузки пользовательских данных для инференса. Это может быть текстовый файл, картинка, аудиофайл или др.
8. Выполнив задание, получите 3 балла.
9. Вы также можете получить дополнительные 2 балла, если реализуете в одном интерфейсе обучение модели и её инференс.

In [140]:
!pip install streamlit

In [196]:
%%writefile main.py

# импортируем необходимые бибилиотеки
import streamlit as st
import tensorflow as tf
import numpy as np
import pandas as pd
from io import StringIO
from keras import models
from keras import layers
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow import keras

df_TransNeft = pd.DataFrame({'select': [0, 1]})

# ввод данных пользователю
st.header("MVP системы прогнозироваия выпуска продукции")

# загрузка ДС
uploaded_file = st.file_uploader("Choose a file")
if uploaded_file is not None:
    data = pd.read_excel(uploaded_file)
    st.write(data)

# обучение модели
l_column, r_column = st.columns(2)

if l_column.button('TRAIN'):
    x_data = data[['Tovar_svar', 'Sborka_chel', 'Svarka_chel', 'Transneft', 'Tovar_nesvar']]
    y_data = data[['Tovar_fact']]
    X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1, random_state=42)
    model = models.Sequential()
    model.add(layers.Dense(10, activation='relu', input_shape=(5,)))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(1, activation='linear'))
    EPOCHS = 10
    BATCH_SIZE = 43
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    history = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.05)
    model.save("/content/OEMZ_1.keras")
    test_accuracy = model.evaluate(X_test, y_test)
    st.write(round(test_accuracy[1], 2))



# загрузка сохраненной модели
model_loaded = tf.keras.models.load_model('/content/OEMZ.keras')

# вывод карты
st.subheader("Производство на карте:")
map_data = pd.DataFrame(
    np.random.randn(1, 2) / [1000, 1000] + [55.016172, 73.195062],
    columns=['lat', 'lon'])

st.map(map_data)

# запрос данных
st.subheader("Введите имеющиеся данные:")

x1 = st.slider('Товарный выпуск свариваемых м/к, тонн', min_value=0, max_value=1500)
x2 = st.slider('Суммарная явочная численность *СБОРЩИКОВ* м/к, чел.', min_value=0, max_value=1000)
x3 = st.slider('Суммарная явочная численность *СВАРЩИКОВ* м/к, чел.', min_value=0, max_value=1000)
x4 = chosen = st.radio('Наличие заказов Транснефть (0 - нет, 1 - да)', df_TransNeft['select'])
x5 = st.slider('Товарный выпуск не свариваемых м/к, тонн', min_value=0, max_value=2500)

st.subheader("Данные в табличном виде:")

data_user = pd.DataFrame({
  'Сварка товар, тн': [x1],
  'Сборщики, чел': [x2],
  'Сварщики, чел': [x3],
  'Транснефть': [x4],
  'Несварные товар, тн': [x5]})

st.write(data_user)

# вывод результата
st.subheader("Прогноз выпуска товарно продукции, тонн")
y_pred = model_loaded.predict(data_user)

left_column, right_column = st.columns(2)

if left_column.button('RESULT'):
    st.write(int(y_pred))

Overwriting main.py


In [197]:
!streamlit run main.py --server.address=localhost >/content/logs.txt & ssh -o "StrictHostKeyChecking no" -R 80:localhost:8501 serveo.net

Forwarding HTTP traffic from https://2e2d8ed230e905abd3d38fa5b45a6399.serveo.net
HTTP request from 95.26.217.10 to https://2e2d8ed230e905abd3d38fa5b45a6399.serveo.net/
HTTP request from 95.26.217.10 to https://2e2d8ed230e905abd3d38fa5b45a6399.serveo.net/static/css/main.a1bc16b2.css
HTTP request from 95.26.217.10 to https://2e2d8ed230e905abd3d38fa5b45a6399.serveo.net/static/media/SourceSansPro-Regular.0d69e5ff5e92ac64a0c9.woff2
HTTP request from 95.26.217.10 to https://2e2d8ed230e905abd3d38fa5b45a6399.serveo.net/static/media/SourceSansPro-SemiBold.abed79cd0df1827e18cf.woff2
HTTP request from 95.26.217.10 to https://2e2d8ed230e905abd3d38fa5b45a6399.serveo.net/static/media/SourceSansPro-Bold.118dea98980e20a81ced.woff2
HTTP request from 95.26.217.10 to https://2e2d8ed230e905abd3d38fa5b45a6399.serveo.net/static/js/main.754d974e.js
HTTP request from 95.26.217.10 to https://2e2d8ed230e905abd3d38fa5b45a6399.serveo.net/_stcore/health
HTTP request from 95.26.217.10 to https://2e2d8ed230e905abd3d